In [1]:
# imports
from utils.config import get_env_var
import requests
from bs4 import BeautifulSoup
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
# get env vars 
API_KEY_GOOGLE = get_env_var("API_KEY_GOOGLE")
SEARCH_ENGINE_ID = get_env_var("SEARCH_ENGINE_ID")
API_GEMINI_KEY= get_env_var("API_GEMINI_KEY")

In [3]:
# the search query you want
query = "learning web developing"
# using the first page
page = 1
# constructing the URL
# doc: https://developers.google.com/custom-search/v1/using_rest
# calculating start, (page=2) => (start=11), (page=3) => (start=21)
start = (page - 1) * 10 + 1
url = f"https://www.googleapis.com/customsearch/v1?key={API_KEY_GOOGLE}&cx={SEARCH_ENGINE_ID}&q={query}&start={start}"

In [4]:
# make the API request
data = requests.get(url).json()

In [5]:
# get the result items
search_items = data.get("items")
# iterate over 10 results found
for i, search_item in enumerate(search_items, start=1):
    try:
        long_description = search_item["pagemap"]["metatags"][0]["og:description"]
    except KeyError:
        long_description = "N/A"
    # get the page title
    title = search_item.get("title")
    # page snippet
    snippet = search_item.get("snippet")
    # alternatively, you can get the HTML snippet (bolded keywords)
    html_snippet = search_item.get("htmlSnippet")
    # extract the page url
    link = search_item.get("link")
    # print the results
    print("="*10, f"Result #{i+start-1}", "="*10)
    print("Title:", title)
    print("Description:", snippet)
    print("Long description:", long_description)
    print("URL:", link, "\n")

========== Result #1 ==========
Title: how you would learn web development if you could start over ? : r ...
Description: Apr 10, 2023 ... Start by gaining a clear understanding of web development concepts, including HTML (Hypertext Markup Language), CSS (Cascading Style Sheets), ...
Long description: Posted by u/StockFan2047 - 215 votes and 181 comments
URL: https://www.reddit.com/r/Frontend/comments/12h51z1/how_you_would_learn_web_development_if_you_could/ 

========== Result #2 ==========
Title: Learn web development | web.dev
Description: Explore our growing collection of courses on key web design and development subjects. An industry expert has written each course, helped by members of the ...
Long description: N/A
URL: https://web.dev/learn 

========== Result #3 ==========
Title: Web Development Courses | Codecademy
Description: Web development courses at Codecademy helps you master HTML, CSS, JavaScript, React, and more. Learn to build web apps and websites.
Long description: W

In [5]:
url = "https://web.dev/learn"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
text = soup.get_text()

lines = text.split('\n')
new_txt = ""
per_words = ["learn" , "course" , "resources"]
nb_line_to_fetch = 4
for line in lines:
    stripped_line = line.strip()
    for word in per_words:
        if word.lower() in stripped_line.lower():
            nb_line_to_fetch = 4
            break
    if stripped_line and nb_line_to_fetch>0:
        new_txt += stripped_line + "\n"
        nb_line_to_fetch -= 1

print(new_txt)

Learn web development  |  web.dev
Skip to main content
Resources
Web Platform
Dive into the web platform, at your pace.
HTML
Learn how to build better user experiences.
Performance
Accessibility
Identity
Learn
Get up to speed on web development.
Learn HTML
Learn CSS
Learn JavaScript
Learn Performance
Learn Accessibility
More courses
Additional resources
Explore content collections, patterns, and more.
AI and the web
Explore
Resources
More
Privacy
Accessibility
Learn
Learn HTML
Learn CSS
Learn JavaScript
Learn Performance
Learn Accessibility
More courses
Additional resources
AI and the web
Explore
PageSpeed Insights
Resources
Privacy
Stay organized with collections
Save and categorize content based on your preferences.
Learn web development
Explore our growing collection of courses on key web design and development subjects. An industry expert has written each course, helped by members of the Chrome team. Follow the modules sequentially, or dip into the topics you most want to learn abo

In [7]:
# Initialize model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=API_GEMINI_KEY,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
 
# System prompt with scraped content
system_prompt = f"""You are a roadmap generator. Based on the topic the human provides, 
generate a structured learning roadmap with:
- Clear sequential steps
- Estimated duration for each step
- Brief description of what to learn in each step
- No additional comments

Format each step as:
Step X: [Topic Name] - [Duration]
Description: [Brief description]

Here is reference material from web.dev to help you create the roadmap:
{new_txt}
"""

messages = [
    ("system", system_prompt),
    ("human", "Web Development"),
]

response = llm.invoke(messages)
print(response.content)

Step 1: HTML Basics - 1 Week
Description: Learn the structure of HTML documents, elements, attributes, and basic tags for creating content.

Step 2: CSS Fundamentals - 1 Week
Description: Understand CSS syntax, selectors, properties, and values for styling HTML elements.

Step 3: JavaScript Basics - 2 Weeks
Description: Learn JavaScript syntax, variables, data types, operators, control flow, and basic DOM manipulation.

Step 4: Responsive Web Design - 1 Week
Description: Learn about media queries, flexible layouts, and responsive images to create websites that adapt to different screen sizes.

Step 5: Version Control with Git - 1 Week
Description: Understand Git concepts, basic commands for version control, and collaboration using platforms like GitHub.

Step 6: Front-End Framework (e.g., React, Angular, or Vue.js) - 4 Weeks
Description: Learn the fundamentals of a front-end framework, including components, state management, and data binding.

Step 7: Back-End Development with Node.js 